In [31]:
from finviz.screener import Screener
import os
import datetime
import pandas as pd
import time
import yfinance as yf
from datetime import datetime
from datetime import timedelta
import math

In [32]:
def fetch_stocks():
    # filters = ['ind_exchangetradedfund','sh_avgvol_o2000']
    filters = ['sh_avgvol_o2000', 'ind_stocksonly', 'cap_largeover', 'sh_price_o10']
    
    stock_list = Screener(filters=filters, table='Performance', order='price')
    
    return stock_list

In [33]:
def cal_VRRS(ticker, ref, rolling_length):
    ticker_sma = ticker.rolling(rolling_length).mean()
    ticker_change = ticker - ticker_sma
    
    ref_sma = ref.rolling(rolling_length).mean()
    ref_change = ref - ref_sma
    
    return (ticker_change/ticker_sma - ref_change/ref_sma) * 100

In [34]:
def get_wr(high, low, close, lookback):
    highh = high.rolling(lookback).max()
    lowl = low.rolling(lookback).min()
    wr = 100 * ((close - highh) / (highh - lowl))
    return wr

In [35]:
def value_to_float(x):
    if type(x) == float or type(x) == int:
        return x
    if 'K' in x:
        if len(x) > 1:
            return float(x.replace('K', '')) * 1000
        return 1000.0
    if 'M' in x:
        if len(x) > 1:
            return float(x.replace('M', '')) * 1000000
        return 1000000.0
    if 'B' in x:
        if len(x) > 1:
            return float(x.replace('B', '')) * 1000000000
        return 1000000000.0
    if 'T' in x:
        if len(x) > 1:
            return float(x.replace('T', '')) * 1000000000000
        return 1000000000000000.0

    return 0.0

In [36]:
ticker_dict = {}

In [37]:
stock_list = fetch_stocks()

100%|██████████████████████████████████████████████████████████████████████████████████| 19/19 [00:07<00:00,  2.70it/s]


In [38]:
start_date = "2017-01-01"
# end_date = "2022-07-12"

interval_time = '1d'

end_date = datetime.today().date()
end_date += timedelta(days = 1)

spy = yf.Ticker("SPY")
spy = spy.history(start=start_date, end=end_date, interval=interval_time)['Close'].dropna()

for stock in stock_list:
    
    ticker = stock['Ticker']
    
    # avg_volume = value_to_float(stock['Avg Volume'])
    # if avg_volume > 2000000:

    stock = yf.Ticker(ticker)    

    stock_close = stock.history(start=start_date, end=end_date, interval=interval_time)['Close'].dropna()
    stock_now = stock.history(start=start_date, end=end_date, interval=interval_time).dropna()
    
    williams = get_wr(stock_now['High'], stock_now['Low'], stock_now['Close'], 2).iloc[-1]

    # Calculate the VRRS indicator:
    rolling_length = 21 

    vrrs = cal_VRRS(stock_close, spy, rolling_length).iloc[-1]
    previous_VRRS = cal_VRRS(stock_close.iloc[:-1], spy.iloc[:-1], rolling_length).iloc[-1]

    difference = vrrs > previous_VRRS
    difference_two = previous_VRRS > vrrs
    
    eDistance = math.dist([vrrs], [previous_VRRS])

    if williams >= -30 and difference and eDistance > 4:
        # print(ticker)
        print(ticker, ' || VRRS: ', round(vrrs, 2), '|| Previous VRRS: ', round(previous_VRRS, 2), ' || Distance', round(eDistance, 2), "|| Short")
    elif williams <= -70 and difference_two and eDistance > 4:
        # print(ticker)
        print(ticker, ' || VRRS: ', round(vrrs, 2), '|| Previous VRRS: ', round(previous_VRRS, 2), ' || Distance', round(eDistance, 2), "|| Long")

SNAP  || VRRS:  2.93 || Previous VRRS:  8.01  || Distance 5.08 || Long
CPNG  || VRRS:  12.81 || Previous VRRS:  18.19  || Distance 5.39 || Long
BEKE  || VRRS:  -10.18 || Previous VRRS:  -1.22  || Distance 8.96 || Long
PLUG  || VRRS:  8.33 || Previous VRRS:  15.47  || Distance 7.14 || Long
PINS  || VRRS:  -4.58 || Previous VRRS:  2.97  || Distance 7.55 || Long
LCID  || VRRS:  2.9 || Previous VRRS:  9.7  || Distance 6.8 || Long
NIO  || VRRS:  -3.12 || Previous VRRS:  5.42  || Distance 8.54 || Long
BILI  || VRRS:  -9.59 || Previous VRRS:  -1.67  || Distance 7.92 || Long
TCOM  || VRRS:  0.45 || Previous VRRS:  6.26  || Distance 5.82 || Long
XPEV  || VRRS:  -2.81 || Previous VRRS:  5.77  || Distance 8.57 || Long
RIVN  || VRRS:  4.05 || Previous VRRS:  11.56  || Distance 7.51 || Long
LVS  || VRRS:  -3.92 || Previous VRRS:  1.09  || Distance 5.01 || Long
SHOP  || VRRS:  -4.66 || Previous VRRS:  2.68  || Distance 7.34 || Long
TWTR  || VRRS:  -13.98 || Previous VRRS:  -5.8  || Distance 8.18 || 